In [ ]:
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import model.utils as utils
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from model.HodgkinHuxley import HodgkinHuxley
from model.HodgkinHuxleyStatsMoments import HodgkinHuxleyStatsMoments
from run_genetic import run_deap

%matplotlib inline

## Set up the model

In [ ]:
seed = 1
seed_p = 2
n_xcorr = 0
n_mom = 4
n_summary = 7
summary_stats = 1
prior_uniform = True
prior_log = False
prior_extent = True
cython=True

true_params, labels_params = utils.obs_params(reduced_model=False)
I, t_on, t_off, dt = utils.syn_current()

obs = utils.syn_obs_data(I, dt, true_params, seed=seed, cython=cython)
obs_stats = utils.syn_obs_stats(data=obs, I=I, t_on=t_on, t_off=t_off, dt=dt, params=true_params,
                                seed=seed, n_xcorr=n_xcorr, n_mom=n_mom, cython=cython,
                                summary_stats=summary_stats,n_summary=n_summary)

LABELS_HH_SUMSTATS = [r'$sp$',r'$rpot$',r'$\sigma_{rpot}$','$m_1$','$m_2$','$m_3$','$m_4$']

# define model, prior, summary statistics and generator
n_processes = 10

seeds_model = np.arange(1,n_processes+1,1)
m = []
for i in range(n_processes):
    m.append(HodgkinHuxley(I, dt, V0=obs['data'][0], reduced_model=False, seed=seeds_model[i],
                           cython=cython,prior_log=prior_log))
p = utils.prior(true_params=true_params,prior_uniform=prior_uniform,
                prior_extent=prior_extent,prior_log=prior_log, seed=seed_p)
s = HodgkinHuxleyStatsMoments(t_on=t_on, t_off=t_off, n_xcorr=n_xcorr, n_mom=n_mom, n_summary=n_summary)
g = dg.MPGenerator(models=m, prior=p, summary=s)
    
bounds = np.asarray([p.lower,p.upper]).T

## pilot run

In [ ]:
seed_inf = 1

pilot_samples = 1000

# network setup
n_hiddens = [50,50]

# convenience
prior_norm = True

# MAF parameters
density = 'maf'
n_mades = 5         # number of MADES

# inference object
res = infer.SNPEC(g,
                obs=obs_stats,
                n_hiddens=n_hiddens,
                seed=seed_inf,
                pilot_samples=pilot_samples,
                n_mades=n_mades,
                prior_norm=prior_norm,
                density=density,
                verbose=True)

## Run genetic algorithm

In [ ]:
algo = 'ibea'
offspring_size = 1000
max_ngen = 100
seed_ibea = 5 # 5,6,7,8,9
final_pop, halloffame, log, hist = run_deap(model=m[0], bounds=bounds, labels_params=labels_params,
                                            summary=s, obs_stats=obs_stats, labels_sum_stats=LABELS_HH_SUMSTATS,
                                            stats_std=res.stats_std,algo=algo, offspring_size=offspring_size,
                                            max_ngen=max_ngen, seed=seed_ibea)

In [ ]:
io.save_pkl((final_pop,halloffame,log,hist),'./results/'+\
            'sim_run_2_offspr'+str(offspring_size)+'_max_gen'+str(max_ngen)+ '_param8_7stats_' + algo + '.pkl')

In [ ]:
final_pop, halloffame, log, hist = io.load_pkl('./results/'+\
                                               'sim_run_2_offspr'+str(offspring_size)+\
                                               '_max_gen'+str(max_ngen)+ '_param8_7stats_' + algo + '.pkl')